# <center>Data Scientist Capstone <center>

Buying a new car is not always the best option. There are many people that like to change cars over the months. Others just don't have enough money to buy a brand new car. There are also cases where someone has just gotten their driver's license and prefers to learn how to drive in a cheaper car.

The second hand market is full of options when buying a second hand car. This market is controlled by the seller. This way, the prices may vary a lot.

My goal is to estimate the price of a car based on certain features. This way, we can know if the car we want to buy is cheap or expensive.
    

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from carbo import*

----------------------------------------
## <center>Extract<center>
  
--------------------------

These dataset came from https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes


This is scraped data of used cars listings. 100,000 listings, which have been separated into files corresponding to each car manufacturer. Each of the `.csv` have listings cars for a certain car brand

In [2]:
audi_df = pd.read_csv('data/audi.csv')
bmw_df = pd.read_csv('data/bmw.csv')
ford_df = pd.read_csv('data/ford.csv')
hyundai_df = pd.read_csv('data/hyundi.csv')
merc_df = pd.read_csv('data/merc.csv')
skoda_df = pd.read_csv('data/skoda.csv')
toyota_df = pd.read_csv('data/toyota.csv')
vauxhall_df = pd.read_csv('data/vauxhall.csv')
vw_df = pd.read_csv('data/vw.csv')



In [3]:
print(audi_df.shape, bmw_df.shape ,ford_df.shape,hyundai_df.shape, merc_df.shape,skoda_df.shape,toyota_df.shape,vauxhall_df.shape,vw_df.shape)

(10668, 9) (10781, 9) (17965, 9) (4860, 9) (13119, 9) (6267, 9) (6738, 9) (13632, 9) (15157, 9)


All the dataframes seem to have the same column size. The features are:

In [4]:
ford_df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType', 'tax',
       'mpg', 'engineSize'],
      dtype='object')

In [5]:
hyundai_df.columns

Index(['model', 'year', 'price', 'transmission', 'mileage', 'fuelType',
       'tax(£)', 'mpg', 'engineSize'],
      dtype='object')

In [6]:
# The tax column in hyndais is different named, I'm going to rename this:
hyundai_df.rename(columns={"tax(£)": "tax"},inplace = True)

I want to merge all the dataframes together, but by doing this, some information is going to be lost, the brand of the model. This is why before merging all dataframes, I'm adding a new column to each dataset: `car_brand`

In [7]:
audi_df['car_brand'] = 'audi'
bmw_df['car_brand'] = 'bmw'
ford_df['car_brand'] = 'ford'
hyundai_df['car_brand'] = 'hyundai'
merc_df['car_brand'] = 'mercedes'
skoda_df['car_brand'] = 'skoda'
toyota_df['car_brand'] = 'toyota'
vauxhall_df['car_brand'] = 'opel'
vw_df['car_brand'] = 'volkswagen'


In [8]:
vw_df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_brand
0,T-Roc,2019,25000,Automatic,13904,Diesel,145,49.6,2.0,volkswagen
1,T-Roc,2019,26883,Automatic,4562,Diesel,145,49.6,2.0,volkswagen
2,T-Roc,2019,20000,Manual,7414,Diesel,145,50.4,2.0,volkswagen
3,T-Roc,2019,33492,Automatic,4825,Petrol,145,32.5,2.0,volkswagen
4,T-Roc,2019,22900,Semi-Auto,6500,Petrol,150,39.8,1.5,volkswagen


In [9]:
df = pd.concat([audi_df, bmw_df,ford_df,hyundai_df,merc_df,skoda_df,toyota_df,vauxhall_df,vw_df], ignore_index=True)

In [10]:
df.shape

(99187, 10)

In [11]:
df.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_brand
0,A1,2017,12500,Manual,15735,Petrol,150,55.4,1.4,audi
1,A6,2016,16500,Automatic,36203,Diesel,20,64.2,2.0,audi
2,A1,2016,11000,Manual,29946,Petrol,30,55.4,1.4,audi
3,A4,2017,16800,Automatic,25952,Diesel,145,67.3,2.0,audi
4,A3,2019,17300,Manual,1998,Petrol,145,49.6,1.0,audi


------------
## <center>TRANSFORM<center>
-------------------
Now I am going to take a look at some of the Nan values, empty and duplicated rows.


In [12]:
for feature in df.columns:
    print('The number of Nan values in the column', feature, 'is:',df[feature].isnull().sum())


The number of Nan values in the column model is: 0
The number of Nan values in the column year is: 0
The number of Nan values in the column price is: 0
The number of Nan values in the column transmission is: 0
The number of Nan values in the column mileage is: 0
The number of Nan values in the column fuelType is: 0
The number of Nan values in the column tax is: 0
The number of Nan values in the column mpg is: 0
The number of Nan values in the column engineSize is: 0
The number of Nan values in the column car_brand is: 0


In [16]:
# check number of duplicates

print('Number of full duplicated rows ( two indentical rows):',df.duplicated().sum())

# But, in this case, since we have nearly 100K rows,
# We shouldn't drop these, since, there is possible to sold the same car ( with the same characteristics) at the same price

Number of full duplicated rows ( two indentical rows): 1475


In [17]:
# Now let's take a look at the year of the car, year above 2022 have to be eliminated
print(df[df['year']>2022])
# We have to drop this row

df = df[df['year']<2023]

         model  year  price transmission  mileage fuelType  tax   mpg  \
39175   Fiesta  2060   6495    Automatic    54807   Petrol  205  42.8   

       engineSize car_brand  
39175         1.4      ford  


In [18]:
df[df['year']>2022]

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,car_brand


## <center> LOAD <center>



In [19]:
df.to_csv('car_sales.csv')